
# Caltech-101 Training Pipeline (Freeze + Unfreeze + 66/33 Split)

Features:
- Google Drive mount
- 66% Train / 33% Test split
- Validation split from training set
- Freeze + Unfreeze training phases
- Early stopping for overfitting control
- Saves best model
- Reports Validation Accuracy and Test Accuracy


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import os

print("GPUs:", len(tf.config.list_physical_devices('GPU')))


In [ ]:

dataset_path = "/content/drive/MyDrive/Caltech101/101_ObjectCategories"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123


In [ ]:

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.33,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.33,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", num_classes)


In [ ]:

train_batches = tf.data.experimental.cardinality(train_ds)
val_size = int(0.1 * train_batches)

val_ds = train_ds.take(val_size)
train_ds = train_ds.skip(val_size)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x,y: (preprocess_input(x), y)).prefetch(AUTOTUNE)
val_ds = val_ds.map(lambda x,y: (preprocess_input(x), y)).prefetch(AUTOTUNE)
test_ds = test_ds.map(lambda x,y: (preprocess_input(x), y)).prefetch(AUTOTUNE)


In [ ]:

data_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])


In [ ]:

base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False


In [ ]:

model = models.Sequential([
    data_aug,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:

checkpoint_path = "/content/drive/MyDrive/best_model_overfit_safe.h5"

callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True)
]


In [ ]:

print("Training with base model frozen...")

history_frozen = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


In [ ]:

print("Fine-tuning with base model unfrozen...")

base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_unfrozen = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)


In [ ]:

plt.plot(history_frozen.history['val_accuracy'], label='Frozen Val Acc')
plt.plot(history_unfrozen.history['val_accuracy'], label='Unfrozen Val Acc')
plt.legend()
plt.title("Validation Accuracy Comparison")
plt.show()


In [ ]:

val_loss, val_acc = model.evaluate(val_ds)
test_loss, test_acc = model.evaluate(test_ds)

print("Validation Accuracy:", val_acc)
print("Test Accuracy:", test_acc)
